In [1]:
import pandas as pd

# 👉🏻 Task 1

In [2]:
df = pd.read_csv('D1.csv')
print(df.info())
# print(df.head(10))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8000 entries, 0 to 7999
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   userId     8000 non-null   int64  
 1   movieId    7958 non-null   float64
 2   rating     8000 non-null   float64
 3   timestamp  8000 non-null   object 
 4   imdbId     7759 non-null   object 
 5   title      8000 non-null   object 
dtypes: float64(2), int64(1), object(3)
memory usage: 375.1+ KB
None


## Task 1-1
What pre-processing was required on the dataset before building the association mining
model? What variables did you include in the analysis? Justify your choice.

In [3]:
# movieId: 
# dropping Nan values first
df = df.dropna(subset=['movieId'])

# float -> int 
df["movieId"] = df["movieId"].astype(int)

In [4]:
# imdbId: 
# dropping Nan values first
df = df.dropna(subset=['imdbId'])

In [15]:
df.info() # for checking

<class 'pandas.core.frame.DataFrame'>
Index: 7726 entries, 0 to 7999
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   userId     7726 non-null   int64  
 1   movieId    7726 non-null   int64  
 2   rating     7726 non-null   float64
 3   timestamp  7726 non-null   object 
 4   imdbId     7726 non-null   object 
 5   title      7726 non-null   object 
dtypes: float64(1), int64(2), object(3)
memory usage: 422.5+ KB


## Task 1-2
Conduct association mining and answer the following:

- a. What ‘min_support’ and `min_confidence’ thresholds were set for this mining
exercise? Rationalize why these values were chosen.
- b. Report the top-5 (interesting) rules and interpret them.

### ⌗ task-1-2-a. ‘min_support’ and `min_confidence’

I set these two values to defualt for now cause I have no clue 😵‍💫

- `min_support` = 0.1
- `min_confidence` = 0 

### ⌗ task-1-2-b. top-5 (interesting) rules and interpret them.

In [6]:
# generating the 
transactions = df.groupby(['userId'])['title'].apply(list)
print(transactions.head(5)) # this output get cut
# for t in transactions[:5]:
#     print(sorted(t))

userId
2    [Interview with the Vampire, Ed Wood, Disclosu...
3    [The White Stripes: Under Great White Northern...
4    [Addams Family Values, Mystery Men, The Fly, F...
5    [Road to Perdition, Almost Famous, Hotel Rwand...
6                                       [Spider-Man 2]
Name: title, dtype: object


In [7]:
# Apriori algorithm
from apyori import apriori
transaction_list = list(transactions)
results = list(apriori(transaction_list))

def convert_apriori_results_to_pandas_df(results):
    rules = []
    for rule_set in results:
        for rule in rule_set.ordered_statistics:
            rules.append([','.join(rule.items_base), ','.join(rule.items_add),
                    rule_set.support, rule.confidence, rule.lift])
    # typecast it to pandas df
    return pd.DataFrame(rules, columns=['Left_side', 'Right_side', 'Support', 'Confidence', 'Lift'])

result_df = convert_apriori_results_to_pandas_df(results)
result_df_sorted = result_df.sort_values(by='Lift', ascending=False)
print(result_df_sorted.head(10))

Empty DataFrame
Columns: [Left_side, Right_side, Support, Confidence, Lift]
Index: []


Oops, didnt get any shit 🥴

So now, we might try to decrease the value of `min_support` to see if there is any notable rules appearing. <br/>
Why? Beacuse `min_support` acts like a threshold for the algorithm to determine which rules should be selected, and the selection criterion is based on the the probability of both A and B apprearing together (e.g. A=>B). <br/>
If the probability is higher than the minimum threshold, the rule is selected.

In [8]:
# set `min_support` to 0.01

results = list(apriori(transaction_list, min_support=0.01))
result_df = convert_apriori_results_to_pandas_df(results)
# sort
result_df_sorted = result_df.sort_values(by='Lift', ascending=False) 
print(result_df_sorted.head(10))

    Left_side             Right_side   Support  Confidence  Lift
0              2001: A Space Odyssey  0.012442    0.012442   1.0
173                            Speed  0.023328    0.023328   1.0
160               Return of the Jedi  0.034215    0.034215   1.0
161                            Rocky  0.012442    0.012442   1.0
162              Saving Private Ryan  0.021773    0.021773   1.0
163                 Schindler's List  0.026439    0.026439   1.0
164                           Scream  0.010886    0.010886   1.0
165                            Se7en  0.027994    0.027994   1.0
166            Sense and Sensibility  0.013997    0.013997   1.0
167              Shakespeare in Love  0.015552    0.015552   1.0


There you go. <br/>

However, the result does not show any valuable insight beacuse the greatest value of "lift" = 1. This means the rules have zero correlation, indicating the two itemsets are independent.

Let's decrease `min_support` a bit more. 

In [14]:
# set `min_support` to 0.006

results = list(apriori(transaction_list, min_support=0.006))
result_df = convert_apriori_results_to_pandas_df(results)
# sort
result_df_sorted = result_df.sort_values(by='Lift', ascending=False) 
print(result_df_sorted.head(10))

                           Left_side                       Right_side  \
655            Rebel Without a Cause                      Rear Window   
654                      Rear Window            Rebel Without a Cause   
624       Ace Ventura: Pet Detective                        The Mummy   
625                        The Mummy       Ace Ventura: Pet Detective   
646                   Twelve Monkeys           Mission: Impossible II   
645           Mission: Impossible II                   Twelve Monkeys   
630                        Apollo 13           The American President   
649                   V for Vendetta  Monty Python and the Holy Grail   
648  Monty Python and the Holy Grail                   V for Vendetta   
631           The American President                        Apollo 13   

      Support  Confidence       Lift  
655  0.006221    0.500000  32.150000  
654  0.006221    0.400000  32.150000  
624  0.006221    0.250000  20.093750  
625  0.006221    0.500000  20.093750  
6

**💡 Interpretation:** <br/>

The result shows high lift in the generated rules, indicating the correlations are high in between the itemsets. <br/>
Taking the first row as an example (lift = 32.15):  <br/>
When a person watches the movie 'Rebel Without a Cause', they are 32 times more likely to also watch the movie 'Rear Window'.

However, since `min_support` is set to a low value (=0.006), this means the generated rules are considered rare, indicating the relationship between the itemsets is sparse. <br/>  

## Task 1-3
List five most watched movies by individuals who have also watched ‘The Shawshank Redemption’ movie.

In [10]:
result_df_sorted.loc[(result_df_sorted['Left_side'].isin(['The Shawshank Redemption'])) | (result_df_sorted['Right_side'].isin(['The Shawshank Redemption']))].head(5)

,Left_side,Right_side,Support,Confidence,Lift
551,,The Shawshank Redemption,0.032659,0.032659,1.0


There is no movie that is found to be watched when The Shawshank Redemption is also watch, because the result is probably filtered out by the `min_support`.

Although I dont see the point of finding the answer for the quesiton,  the code below can calculate the most watched movies base on the transactions:


#### *- Hide below -*

In [11]:
from collections import Counter

lists_of_movies = []
for t in transactions:
    if 'The Shawshank Redemption' in t:
        lists_of_movies.append(t)
        

# Flatten the lists of movies
all_movies = [movie for sublist in lists_of_movies for movie in sublist]

# Count the occurrences of each movie
movie_counter = Counter(all_movies)

# Remove 'The Shawshank Redemption' from the movie counter
del movie_counter['The Shawshank Redemption']

# Filter movies watched by individuals who have watched 'The Shawshank Redemption'
filtered_movies = {movie: count for movie, count in movie_counter.items() if movie in all_movies}

# Sort movies by the number of times they were watched
sorted_movies = sorted(filtered_movies.items(), key=lambda x: x[1], reverse=True)

# Get the top five most watched movies
top_five_movies = sorted_movies[:5]

# Print the result
print("Five most watched movies by individuals who have also watched 'The Shawshank Redemption':")
for movie, count in top_five_movies:
    print(f"{movie}: {count} times")        

Five most watched movies by individuals who have also watched 'The Shawshank Redemption':
Jurassic Park: 3 times
The Client: 3 times
Stargate: 3 times
The Lord of the Rings: The Fellowship of the Ring: 3 times
Citizen Kane: 2 times


#### *- hide above -*

## Task 1-4
Can you perform sequence analysis on this dataset? If yes, present your results. If not,
rationalize why.

As aforementioned, since the relationship between data is quite sparse, it might not be recommended to apply sequence analysis to this dataset as it will only result in sparser relationship, making it harder to find more insights.  

## Task 1-5
In what ways can the results of this task be utilized by the relevant decision-makers?

The results show high correlations (high lift) between data in a small number of cases (low support), meaning that there are some ways we can utitlise this result to make decision: 
- Improve the niche market by promoting the movies that are highly correlated together.
- Some movies are found very old (made in 1980-2000). Ｗe may aim to target the market.
- Make a decision on taking more variables (e.g. rating or genre if it's possible) into account so that we can create more insight from the data